In [ ]:
import sys
# setting path
sys.path.append('..')
# importing
import ECC_library
from qiskit.circuit import Gate
from qiskit import ClassicalRegister, QuantumCircuit, QuantumRegister
from qiskit import Aer, execute, IBMQ, BasicAer
from qiskit.aqua import QuantumInstance

In [ ]:
def controlled_addition_without_carry(circuit,ctrl,x,y,N):
    sub_x = QuantumRegister(n)
    sub_y = QuantumRegister(n)
    sub_circ = QuantumCircuit(sub_x,sub_y,name='add_no_car')
    ECC_library.addition_without_carry(sub_circ,sub_x,sub_y,n)
    sub_inst = sub_circ.to_gate().control()
    circuit.append(sub_inst,ctrl[0:1]+x[0:n]+y[0:n])

In [ ]:
def controlled_comparator(circuit,ctrl,x,y,c,n):
    ECC_library.controlled_subtraction(circuit,ctrl,y,x,c,n)
    controlled_addition_without_carry(circuit,ctrl,y,x,n)

In [ ]:
def controlled_addition_modp(circuit,ctrl,x,y,c,g,p,n):
    p1 = p+'0'
    ECC_library.controlled_addition(circuit,ctrl,x,y,c,n)
    ECC_library.constant_subtraction(circuit,y[0:n]+c[0:1],g,p1,n+1)
    ECC_library.controlled_constant_addition(circuit,c,y,g,p,n)
    controlled_comparator(circuit,ctrl,y,x,c,n)
    circuit.x(c)

In [ ]:
def test(a,b,p,n,control,g_s):
    ctrl = QuantumRegister(1,name='ctrl')
    x = QuantumRegister(n,name='x')
    y = QuantumRegister(n,name='y')
    c = QuantumRegister(1,name='d')
    g = QuantumRegister(1,name='g')
    clas_a = ClassicalRegister(n)
    clas_b = ClassicalRegister(n)
    clas_c = ClassicalRegister(1)
    clas_g = ClassicalRegister(1)
    circuit = QuantumCircuit(ctrl,x,y,c,g,clas_a,clas_b,clas_c,clas_g)
    s_a=bin(int(a))[2:].zfill(n)
    s_b=bin(int(b))[2:].zfill(n)
    s_p=bin(int(p))[2:].zfill(n)
    s_p=s_p[::-1]
    for i in range(0, n):
        if s_a[i] == '1':
            circuit.x(x[n-1-i])
        if s_b[i] == '1':
            circuit.x(y[n-1-i])
    if control=='1':
        circuit.x(ctrl)
    if g_s=='1':
        circuit.x(g)
    controlled_addition_modp(circuit,ctrl,x,y,c,g,s_p,n)
    for i in range(0, n):
        circuit.measure(x[i],clas_a[i])
    for i in range(0, n):
        circuit.measure(y[i],clas_b[i])
    circuit.measure(c,clas_c)
    circuit.measure(g,clas_g)
    print('a:',a,'b:',b,'p:',p,'n:',n,'g:',g_s,'ctrl:',control)
    backend = BasicAer.get_backend('qasm_simulator')
    simulation = execute(circuit, backend=backend ,shots=10)
    sim_result = simulation.result()
    out=list(sim_result.get_counts().keys())[0]
    output=out.split(' ')
    output_a = output[3]
    output_b = output[2]
    output_c = output[1]
    res_g = output[0]
    res_a = int(output_a, 2)
    res_b = int(output_b, 2)
    res_c = int(output_c, 2)
    print('res_a:',res_a,'res_b:',res_b,'res_c:',res_c,'res_g:',res_g)
    if ((res_b!=(a+b)%p and control=='1') or (res_b!=b and control=='0') or res_c!=0 or res_g!=g_s or res_a!=a):
        print('found!')

In [ ]:
for n in range (4,5):
    for p in range (7,9,2):
        for i in range (0,p):
            for j in range (0,p):
                test(i,j,p,n,'0','0')
                test(i,j,p,n,'0','1')
                test(i,j,p,n,'1','0')
                test(i,j,p,n,'1','1')
                print('-----------')